In [1]:
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
pd.set_option('display.max_colwidth', 200)

def toFloat(df,cols):
    for col in cols:
        df[col] = pd.to_numeric(df[col],errors='coerce')

def toInt(df,cols):
    for col in cols:
        df[col] = pd.to_numeric(df[col],errors='coerce')
        df[col] = df[col].fillna(0).astype(np.int64)

In [2]:
major = pd.read_csv("Most-Recent-Cohorts-Field-of-Study.csv")
major.shape

(233979, 160)

In [3]:
major.replace('PrivacySuppressed', np.NaN, inplace=True)

```
major[major['CREDLEV']==3][['INSTNM','CIPDESC','EARN_NE_MDN_3YR','EARN_COUNT_NE_3YR','EARN_MDN_4YR','EARN_COUNT_WNE_4YR']].dropna(axis=0, thresh=4).to_csv("reportcard_major_earning.csv", index=False)
```
```
major[major['INSTNM'].str.startswith('Washington University') & major['CIPDESC'].str.startswith('Computer')].to_csv("washu_reportcard_major.csv", index=False)
```

In [4]:
major.dropna(axis=0, thresh=30, inplace=True) # drop rows (axis=0) if less than 30 attributes have values (non-null/NaN)
major.drop(columns=['OPEID6', 'INSTNM','CONTROL','MAIN'], inplace=True)
major.shape

(56602, 156)

In [5]:
datadict_m = pd.read_csv("CollegeScorecardDataDictionary_Major.csv")
datadict_m.replace({'cip_4_digit':'cip'},regex=True,inplace=True)
code2name = dict(zip(datadict_m['VARIABLE NAME'], datadict_m['developer-friendly name']))
major.rename(columns = code2name, inplace = True)

In [6]:
major.head()

,cip.unit_id,cip.code,cip.title,cip.credential.level,cip.credential.title,cip.counts.ipeds_awards1,cip.counts.ipeds_awards2,cip.debt.staff_grad_plus.all.all_inst.count,cip.debt.staff_grad_plus.all.all_inst.average,cip.debt.staff_grad_plus.all.all_inst.median,...,cip.repayment.4_yr_bb_fed_comp.count,cip.repayment.4_yr_bb_fed_comp.default,cip.repayment.4_yr_bb_fed_comp.delinquent,cip.repayment.4_yr_bb_fed_comp.forbearance,cip.repayment.4_yr_bb_fed_comp.deferment,cip.repayment.4_yr_bb_fed_comp.noprogress,cip.repayment.4_yr_bb_fed_comp.makingprogress,cip.repayment.4_yr_bb_fed_comp.fullypaid,cip.repayment.4_yr_bb_fed_comp.discharge,cip.distance
15,100654.0,1002,Audiovisual Communications Technologies/Technicians.,3,Bachelor's Degree,35.0,23.0,45,NaN,36250,...,16,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
16,100654.0,1101,"Computer and Information Sciences, General.",3,Bachelor's Degree,31.0,20.0,41,NaN,31000,...,29,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1
24,100654.0,1312,"Teacher Education and Professional Development, Specific Levels and Methods.",5,Master's Degree,19.0,13.0,NaN,NaN,NaN,...,33,NaN,NaN,0.60 - 0.79,NaN,NaN,NaN,NaN,NaN,2
25,100654.0,1313,"Teacher Education and Professional Development, Specific Subject Areas.",3,Bachelor's Degree,2.0,0.0,NaN,NaN,NaN,...,38,NaN,NaN,0.60 - 0.79,NaN,NaN,NaN,NaN,NaN,1
29,100654.0,1410,"Electrical, Electronics and Communications Engineering.",3,Bachelor's Degree,34.0,33.0,50,NaN,35000,...,22,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,1


In [7]:
toInt(major,[v for v in list(major.columns) if "count" in v]) # some 'count' columns are not integer

In [46]:
major.to_csv("reportcard_major.csv", index=False)

## Reportcard.csv

In [8]:
df = pd.read_csv("Most-Recent-Cohorts-Institution.csv")
df.shape

/tmp/ipykernel_11164/4140189609.py:1: DtypeWarning: Columns (9,1537,1540,1542,1606,1608,1614,1615,1619,1620,1621,1622,1623,1624,1625,1626,1627,1628,1629,1703,1704,1725,1726,1727,1728,1729,1743,1815,1816,1817,1818,1823,1824,1830,1831,1879,1880,1881,1882,1883,1884,1885,1886,1887,1888,1889,1890,1891,1892,1893,1894,1895,1896,1897,1898,1909,1910,1911,1912,1913,1957,1958,1959,1960,1961,1962,1963,1964,1965,1966,1967,1968,1969,1970,1971,1972,1973,1974,1975,1976,1983,1984,2376,2377,2403,2404,2495,2496,2497,2498,2499,2500,2501,2502,2503,2504,2505,2506,2507,2508,2509,2510,2511,2512,2513,2514,2515,2516,2517,2518,2519,2520,2521,2522,2523,2524,2525,2526,2527,2528,2529,2530,2958,3215,3231) have mixed types. Specify dtype option on import or set low_memory=False.
  df = pd.read_csv("Most-Recent-Cohorts-Institution.csv")


(6543, 3232)

In [9]:
### Adding %Tech %Busines
df["prc_tech"] = (df['PCIP11'] + df['PCIP14'] + df['PCIP15'] + df['PCIP27'] + df['PCIP40'] + df['PCIP41'])
df["prc_business"] = df['PCIP52']
df["prc_biology"] = df['PCIP26']
df["prc_stem"] = df["prc_tech"] + df["prc_business"] + df["prc_biology"]

df.dropna(axis=0, thresh=1500, inplace=True) # drop rows (axis=0) if less than 1500 attributes have values (non-null/NaN)
df.shape

(6357, 3236)

In [10]:
df.shape
df.replace('PrivacySuppressed', np.NaN, inplace=True)
df.dropna(axis=1, thresh=500, inplace=True) # drop columns (axis=1) if less than 500 entries have values (non-null/NaN)
df.shape

/tmp/ipykernel_11164/3490906837.py:2: FutureWarning: Downcasting behavior in `replace` is deprecated and will be removed in a future version. To retain the old behavior, explicitly call `result.infer_objects(copy=False)`. To opt-in to the future behavior, set `pd.set_option('future.no_silent_downcasting', True)`
  df.replace('PrivacySuppressed', np.NaN, inplace=True)


(6357, 2774)

In [11]:
df = df[(df['CCBASIC'] > 0) & (df['CCSIZSET'] > 2)].copy() # Carnegie Classification for 2 & 4 year college only
print(df.shape)

(2996, 2774)


In [12]:
toInt(df,['UGDS','FAMINC','MD_FAMINC','MD_EARN_WNE_P10','PCT25_EARN_WNE_P6','PCT25_EARN_WNE_P10','RPY_3YR_RT_SUPP',
    'PELL_RPY_3YR_RT_SUPP','MN_EARN_WNE_P10', 'PCT90_EARN_WNE_P6','PCT90_EARN_WNE_P10'])

In [13]:
toFloat(df,['FAMINC','MD_FAMINC','MD_EARN_WNE_P10','PCT25_EARN_WNE_P6','PCT25_EARN_WNE_P10','RPY_3YR_RT_SUPP','PELL_RPY_3YR_RT_SUPP',
            'PCT_BLACK','PCT_WHITE','PCT_HISPANIC','PCT_ASIAN','MN_EARN_WNE_P10','PCT90_EARN_WNE_P6','PCT90_EARN_WNE_P10'])

In [14]:
datadict = pd.read_csv("CollegeScorecardDataDictionary.csv")
datadict.replace({'by_income_level':'income', '.working_not_enrolled':'','outcome_percentage':'perc'},regex=True,inplace=True)
code2name = dict(zip(datadict['VARIABLE NAME'], datadict['developer-friendly name']))
df.rename(columns = code2name, inplace = True)
df.rename(columns = {'location.lon':'lon', 'location.lat':'lat'}, inplace = True)
regionMap = {0:"U.S. Service Schools",1:"New Englend",2:"Mid East",3:"Great Lakes",4:"Plains",5:"Southeast",6:"Southwest",7:"Rocky Mountains",8:"Far West",9:"Outlying"}
df['region_id'] = df['region_id'].map(regionMap)
localeMap = {11:'Large City',12:'Midsize City',13:'Small City',21:'Large Suburb',22:'Midsize Suburb',23:'Small Suburb',
        31:'Fringe Town',32:'Distant Town',33:'Remote Town',41:'Fringe Rural',42:'Distant Rural',43:'Remote Rural'}
df['locale'] = df['locale'].map(localeMap)
df['net_price.income.110001-plus']=df[['net_price.public.income.110001-plus','net_price.private.income.110001-plus']].max(axis=1)


```df[(df['degrees_awarded.predominant'] == 3) & (df['ownership'] == 3)].to_csv("forprofit_4yr_colleges.csv", index=False) # for profit 4-yr colleges
df[(df['degrees_awarded.predominant'] == 3) & (df['ownership'] != 3) & (df['size'] < 500)].to_csv("small_4yr_colleges.csv", index=False) # small non-profit 4-yr

df[(df['degrees_awarded.predominant'] == 3) & (df['ownership'] != 3) & (df['size'] >= 500)].to_csv("reportcard.csv", index=False) # Predominantly bachelor's-degree granting
df[df['degrees_awarded.predominant'] == 2].to_csv("reportcard_cc.csv", index=False) # Predominantly associate's-degree granting```

In [16]:
df[(df['degrees_awarded.predominant'] == 3) & (df['ownership'] != 3) & (df['size'] >= 500)].to_csv("reportcard.csv", index=False)

In [17]:
df.shape

(2996, 2775)

In [30]:
mDict = pd.concat([datadict_m[['NAME OF DATA ELEMENT','developer-friendly name']],
           datadict[['NAME OF DATA ELEMENT','developer-friendly name']]])\
        .rename(columns = {"NAME OF DATA ELEMENT":"Description","developer-friendly name":"Name"})
        

In [39]:
mDict[mDict['Name'].notnull()].to_csv("dict.csv", index=False)

### Merge

In [43]:
l = list(major['cip.title'].unique())
l.insert(0,'---')

In [53]:
major.shape

(47847, 117)

In [55]:
df.merge(major[major['cip.title']=='Computer Science.'], left_on='id', right_on='cip.unit_id').shape

(299, 2671)

## Graph

In [ ]:
import seaborn as sns

d1 = df[['CCBASIC','CCSIZSET']]
for i, column in enumerate(d1.columns[:]):
    sns.displot(d1[column], label=column)

# Add a legend and show the plot
plt.legend()
plt.show()

## More Processing

no admission rate, or the rate is obviously wrong < 2%
```
c = df['ADM_RATE'].isnull() | df['ADM_RATE'].le(0.02, fill_value=1.0) 
df = df[~c] # exclude colleges that doon't provide admin rate
```
fields with their "Null count"
```
df.isnull().sum().to_csv("null_fields.csv")
```

In [ ]:
# regionMap = {0:"U.S. Service Schools",1:"New Englend",2:"Mid East",3:"Great Lakes",4:"Plains",5:"Southeast",6:"Southwest",7:"Rocky Mountains",8:"Far West",9:"Outlying"}
# df['REGION'] = df['REGION'].map(regionMap)

In [ ]:
selected = ['OPEID6','INSTNM','UGDS','INSTURL','LATITUDE','LONGITUDE','CITY','STABBR','ZIP','PREDDEG','REGION','ADM_RATE','MN_EARN_WNE_P10','MD_EARN_WNE_P10',
            'PCT90_EARN_WNE_P6','PCT90_EARN_WNE_P10','PCT25_EARN_WNE_P6','PCT25_EARN_WNE_P10',
            'NPT45_PRIV','NPT45_PUB','NPT43_PRIV','NPT43_PUB','NPT41_PRIV','NPT41_PUB','NPT4_PUB','NPT4_PRIV',
            'FAMINC','MD_FAMINC','RPY_3YR_RT_SUPP','PELL_RPY_3YR_RT_SUPP','CDR3',
            'PCIP11','PCIP15','PCIP27','PCIP40','PCT_WHITE','PCT_BLACK','PCT_HISPANIC','PCT_ASIAN',
            'UGDS_WHITE','UGDS_ASIAN','UGDS_BLACK','UGDS_HISP','C150_4_BLACK','C150_4_HISP','C150_4_ASIAN','C150_4_WHITE',
            'Tech Degrees','Business Degree','Biology Degree','STEM Degrees']

In [ ]:
display = {'INSTNM':'College','UGDS':'Undergraduate Enrollment','PREDDEG':'Predominant degree','ADM_RATE':'Admission rate',
           'MN_EARN_WNE_P10':'Mean Earnings 10Yr','MD_EARN_WNE_P10':'Median Earnings 10Yr',
            'PCT90_EARN_WNE_P6':'90% earnings 6Yr','PCT90_EARN_WNE_P10':'90% earnings 10Yr','PCT25_EARN_WNE_P6':'25% earnings 6Yr',
            'PCT25_EARN_WNE_P10':'25% earnings 10Yr','NPT45_PRIV':'Net Price 110k family (Private)','NPT45_PUB':'Net Price 110k family (Public)',
            'NPT43_PRIV':'Net Price 48-75k family (Private)','NPT43_PUB':'Net Price 48-75k family (Public)',
            'NPT41_PRIV':'Net Price 0-30k family (Private)','NPT41_PUB':'Net Price 0-30k family (Public)',
            'NPT4_PUB':'Net Price All Income (Public)','NPT4_PRIV':'Net Price All Income (Private)',
            'FAMINC':'Avg Family Income','MD_FAMINC':'Median Family Income','RPY_3YR_RT_SUPP':'3Yr Repayment Rate',
            'PELL_RPY_3YR_RT_SUPP':'3Yr Repayment Rate (Pell Students)','CDR3':'3Yr Default Rate',
            'PCIP11':'% CS/IT','PCIP14':'% Engineering','PCIP15':'% Engineering Related','PCIP27':'% Math/Stats','PCIP40':'% Physical Science',
            'PCT_WHITE':'% students neighbors Whites','PCT_BLACK':'% students neighbors Blacks','PCT_HISPANIC':'% students neighbors Hispanic',
            'PCT_ASIAN':'% students neighbors Asians','UGDS_WHITE':'% undergrades Whites','UGDS_ASIAN':'% undergrades Asians',
            'UGDS_BLACK':'% undergrades Blacks','UGDS_HISP':'% undergrades Hispanic','C150_4_BLACK':'6Yr Completion % Blacks',
            'C150_4_HISP':'6Yr Completion % Hispanics','C150_4_ASIAN':'6Yr Completion % Asians','C150_4_WHITE':'6Yr Completion % Whites'}

In [ ]:
s1 = df[selected].isnull().sum()
s2 = df[selected].dtypes
s3 = s2.index.to_series().map(code2name)
s1.name = "nullcount"
s2.name = "dtypes"
s3.name = "desc"
info = pd.concat([s1, s2, s3], axis=1)
info
#info.to_csv("lookup.csv")

In [ ]:
out = df[selected].copy()
out.set_index('OPEID6', inplace=True)
out.rename(columns=display, inplace=True)

In [ ]:
out['Net Price (All Income)'] = out['Net Price All Income (Public)'].fillna(0) + out['Net Price All Income (Private)'].fillna(0)
out['Net Price 0-30k'] = out['Net Price 0-30k family (Public)'].fillna(0) + out['Net Price 0-30k family (Private)'].fillna(0)
out['Net Price 110K+'] = out['Net Price 110k family (Public)'].fillna(0) + out['Net Price 110k family (Private)'].fillna(0)

In [ ]:
df.dropna(df['LONGITUDE'].isnull() | df['LATITUDE'].isnull()]

In [ ]:
mrc_display = {'tier_name':'Tier Name', 'female':'% Female','k_married':'% Married','mr_kq5_pq1':'Mobility rate (80%->20%)',
               'mr_ktop1_pq1':'Upper-tail mobility rate (80%->1%)','par_mean':'Mean parental income','par_median':'Median parent household income',
               'par_rank':'Mean parental income rank','k_rank':'Mean kid earnings rank','k_mean':'Mean kid earnings',
               'k_median':'Median child individual earnings'}

In [ ]:
mrc = pd.read_csv("mrc_table2.csv")
mrc.rename(columns={'super_opeid':'OPEID6'}, inplace=True)
mrc.set_index('OPEID6', inplace=True)
mrc.rename(columns=mrc_display, inplace=True)

In [ ]:
out.join(mrc, how='inner').to_csv("reportcard.csv")